## **_Integrating OAuth:_**

-   This notebook contains the steps I used to incorporate authentication strategies to this project.

**_re: Angela's WebDev Course Section 35. Authentication and Security_**

**_Will be referencing `"/.projectReference\oauthReference.ipynb"`._**

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

### **_pgAdmin Stuff:_**

-   If you've forgotten the names of tables in a db...

<font size=1>

```sql
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
AND table_type = 'BASE TABLE';
```

</font>


-   Looking up current `secrets` db table `users` state:

<font size=1>

```sql
SELECT * FROM users;
```

</font>

-   Look at up `js_capstone_book` db's table `book_recommendations`:

<font size=1>

```sql
SELECT * FROM book_recommendations;
```

</font>

-   We need to now enter columns for user

-   Add new columns:

```sql
ALTER TABLE book_recommendations
ADD user_name VARCHAR(100),
ADD password VARCHAR(100);
```

-   Success! New columns added...

-  Change `user_name` to `user_email` in case we want to add an actual username or have the user's actual name in the db.


```sql
ALTER TABLE book_recommendations
RENAME user_name to user_email;
```

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

### **_Imports/Install:_**

<font size=1>

```js
import bcrypt from "bcrypt";
import session from "express-session";
import passport from "passport";
import { Strategy as LocalStrategy } from "passport-local";
import { Strategy as GoogleStrategy } from "passport-google-oauth20";
```

</font>

<font size=1>

```console
≫  npm i bcrypt passport express-session passport-local passport-google-oauth20
≫ npm update
≫ nodemon index.js
```

</font>

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

### **_Preliminary Stuff to Integrate:_**

-   Add environment variables in file `.env`.

<font size=1>

```js
app.use(
    session({
        secret: process.env.SESSION_SECRET,
        resave: false,
        saveUninitialized: true,

        cookie: { maxAge: 1000 * 60 * 60 * 24 },
    })
);
```

</font>

<font size=1>

```js
app.use(passport.initialize());
app.use(passport.session());
```

</font>

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

### **_Getting HTML/EJS in Order:_**

-   Convert syntax/classes from yuAuth lesson to match jsCapstoneBook project.

-   **ASIDE:** I modified `index.js` to allow adding the book with just the title and whatever the user entered (except author) in case the book isn't found online. Added a blank book image for these cases. `openlibrary.org` seems to be down...maybe the `archive.org` hack??

-   Moved a couple of functions from `index.js` to `utilityFunctions.js`, figured out how to export/import/implement. `openlibrary.org` is still down! 101624

-   Finished aligning/formatting .ejs files with css. Register and Login pages now looking good!

-   CSS formatting for top links and text for Google link.

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

### **_`index.js`:_**

-   Added `if (req.isAuthenticated())` requirement to index `"/"` route to allow viewing of (adding to) book list only if user is registered/signed in.

-   Changed main page to `login` and what was `"/"` to `"/booklist"` to facilitate ease of routing, especially re:

<font size=1>

```js
app.get(
    "/auth/google/booklist",
    passport.authenticate("google", {
        successRedirect: "/booklist",
        failureRedirect: "/login",
    })
);
```

</font>

-   Actually, changed it back. `index.ejs` should be the book list page, and located at `"/"`!

-   Got the rest of the JavaScript transferred over from the Angela's `secrets` lesson.

-   Next step to make sure it runs smoothly after creating instance for this project on Google Cloud...

    -   Review lesson where Angela walks us through steps to register new project on Google.

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

-   Create project and set up Google OAuth functionality via https://cloud.google.com.

-   Integrate values into environment.

-   Make sure endpoints/callbacks are correct in `index.js`.

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

-   Now realizing I need a new table for `users`, separate from `book_recommendations`.

-   Later make them relational.

<font size=1>

```sql
ALTER TABLE book_recommendations
DROP COLUMN user_email,
DROP COLUMN password;
```

</font>

<font size=1>

```sql
CREATE TABLE users (
	id SERIAL PRIMARY KEY,
	user_email VARCHAR(100) NOT NULL UNIQUE,
	passsword VARCHAR(100)
)
```

</font>

-   Oopsie!

<font size=1>

```sql
ALTER TABLE users
RENAME COLUMN passsword TO password;
```

</font>

-   On the right track. User can register and edit book list, but cannot login!

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

### **_Duplicate Database:_**

-   So it has same name as this project.


<font size=1>

```sql
CREATE DATABASE js_expr_oauth_db
WITH TEMPLATE js_capstone_book;
```

</font>

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

-   I'd modified the `session()` code to:

<font size=1>

```js
app.use(
    session({
        // Keeps session data secure:
        secret: process.env.SESSION_SECRET,
        // Prevents session from being saved back to
        //  the store if it hasn't changed:
        resave: false,
        // Save uninitialized sessions (new but
        //  not modified):
        saveUninitialized: true,
        cookie: {
            // One day expiration time for the cookie:
            maxAge: 1000 * 60 * 60 * 24,
            // Ensures the cookie is sent only over HTTPS:
            secure: true,
            // Ensures the cookie can be sent in cross-
            //  site requests (for OAuth):
            sameSite: "None",
        },
    })
);
```

</font>

-   When I was having issues with OAuth.

-   Having fixed the issues by fixing the URI endpoint, I'm going back to the original version, as the above causes no Google screen to appear on Sign In:

```js
app.use(
    session({
        secret: process.env.SESSION_SECRET,
        resave: false,
        saveUninitialized: true,
        cookie: { maxAge: 1000 * 60 * 60 * 24 },
    })
);
```

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

-   openlibrary.org is back up!

-   After signing in with Google OAuth, the add book form works great.

-   OAuth register redirects to login: look into this later, not super important.

-   Logout working fine, `req.isAuthenticated()` returning `false`, all good (for Google sign in).

-   Local Register is working and redirects to book list. Going to `/` redirects user to `/books`, that's good.

-   Local login doesn't do anything.

-   Arrgh -- it turns out the issue was `new LocalStrategy(async function verify(username, password, cb)` requires `password`. I'd wanted to use `userEmail`. If you don't specify a different name like this:

```js
app.post("/", passport.authenticate('local', {
    usernameField: 'userEmail',  // Add this
    passwordField: 'password',    // This is default, but shown for completeness
    successRedirect: '/books',
    failureRedirect: '/'
}));
```

...then you need to use the default param here and in the ejs.

-   I'll just use `username` for now...

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

<font size=1>

```sql
-- sql
```

</font>

<font size=1>

```console
≫ console
← 
```

</font>

<font size=1>

```js
// js
```

</font>

<font size=1>

```sql
-- sql
```

</font>

<font size=1>

```console
≫ console
← 
```

</font>

<hr style="height: 0; box-shadow: 0 0 5px 4px crimson; width: 95%;">

<hr style="height: 0; box-shadow: 0 0 5px 4px dodgerblue; width: 85%;">